In [ ]:
import sys, time, os, asyncio, glob

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
#%matplotlib widget
import pickle as pkl
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta

import asyncio
from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

from lsst_efd_client import EfdClient

In [ ]:
# Just using this in the notebook for now.
def merge_packed_PSD(packed_dataframe, base_field, sensor_names):
    """Select fields that represent the Power Spectral Density of \                                                          
    a sensor and unpack them into a dataframe with PSD vs frequency.                                                         
    Parameters                                                                                                               
    ----------                                                                                                               
    packed_dataframe : `pandas.DataFrame`                                                                                    
        packed data frame containing the desired data                                                                        
    base_field :  `str`                                                                                                      
        Base field name that will be expanded to query all                                                                   
        vector entries.                                                                                                      
    sensor_name :  `str` or list                                                                                             
        Name of the sensor(s) of interest.                                                                                   
    Returns                                                                                                                  
    -------                                                                                                                  
    result : `pandas.DataFrame`                                                                                              
        A `pandas.DataFrame` containing the results of the query.                                                            
    """
    minPSDFrequency = packed_dataframe['minPSDFrequency'][0]
    maxPSDFrequency = packed_dataframe['maxPSDFrequency'][0]
    numDataPoints = packed_dataframe['numDataPoints'][0]
    if isinstance(sensor_names, str):
        sensor_names = [sensor_names, ]

    packed_dataframe = packed_dataframe.loc[packed_dataframe.sensorName.isin(sensor_names)]
    packed_fields = [k for k in packed_dataframe.keys()
                     if k.startswith(base_field) and k[len(base_field):].isdigit()]
    packed_fields = sorted(packed_fields, key=lambda k: int(k[len(base_field):]))  # sort by pack ID                         
    npack = len(packed_fields)
    assert numDataPoints == npack, "Number of packed data points does not match numDataPoints!"
    packed_len = len(packed_dataframe)
    output = np.empty(npack * packed_len)
    deltaF = float(maxPSDFrequency - minPSDFrequency) / (npack - 1) # Frequency step                                         
    columns = []
    for i in range(npack):
        label = f"{base_field}{i}"
        columns.append(minPSDFrequency + i * deltaF)
        output[i::npack] = packed_dataframe[label]
    output = np.reshape(output, (packed_len, npack))
    return pd.DataFrame(data=output, columns=columns, index=packed_dataframe.index)

# This dictionary defines the axis scramble
trueAxes = {'AuxTel-M1':{'X':'El', 'Y':'Az', 'Z':'Opt'}, \
            'AuxTel-M2':{'X':'El', 'Y':'Az', 'Z':'Opt'}, \
            'AuxTel-Truss':{'X':'El', 'Y':'Opt', 'Z':'Az'}}

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
# Times to start looking at data
center_time = Time("2022-10-04 08:00:00Z", scale='utc')
start = center_time - TimeDelta(30.0, format='sec') 
end = center_time + TimeDelta(30.0, format='sec') 

In [ ]:
ux = await client.select_time_series('lsst.sal.ESS.airTurbulence', \
                                                ['ux'],  start, end)
uy = await client.select_time_series('lsst.sal.ESS.airTurbulence', \
                                                ['uy'],  start, end)
uz = await client.select_time_series('lsst.sal.ESS.airTurbulence', \
                                                ['uz'],  start, end)

In [ ]:
# Smooth the data with a rolling average
ux_vals = np.array(ux.values[:, 0])
uy_vals = np.array(uy.values[:, 0])
uz_vals = np.array(uz.values[:, 0])
ux_mean = ux_vals.mean()
ux_std = ux_vals.std()
uy_mean = uy_vals.mean()
uy_std = uy_vals.std()
uz_mean = uz_vals.mean()
uz_std = uz_vals.std()
rolling = 10
ux_roll = ux.rolling(rolling).sum() / rolling
uy_roll = uy.rolling(rolling).sum() / rolling
uz_roll = uz.rolling(rolling).sum() / rolling

In [ ]:
ymax = 2.0
xplot = (center_time - TimeDelta(10.0, format='sec')).isot
plt.figure(figsize=(8,4))
plt.subplots_adjust(wspace=0.5)
plt.suptitle("AuxTel Anemometer Summary - 20221004")
plt.subplot(1,3,1)
plt.title("UX")
ux_roll['ux'].plot()
plt.text(xplot, ymax*0.8, f"Mean={ux_mean:.2f}")
plt.text(xplot, ymax*0.7, f"Std={ux_std:.2f}")
plt.ylim(-ymax,ymax)
plt.subplot(1,3,2)
plt.title("UY")
uy_roll['uy'].plot()
plt.text(xplot, ymax*0.8, f"Mean={uy_mean:.2f}")
plt.text(xplot, ymax*0.7, f"Std={uy_std:.2f}")
plt.ylim(-ymax,ymax)
plt.subplot(1,3,3)
plt.title("UZ")
uz_roll['uz'].plot()
plt.text(xplot, ymax*0.8, f"Mean={uz_mean:.2f}")
plt.text(xplot, ymax*0.7, f"Std={uz_std:.2f}")
plt.ylim(-ymax,ymax)
#plt.savefig("/home/craiglagegit/DATA/Anemometer_3D_29Sep22.pdf")

In [ ]:
#Accelerometers on a common scale
accel_data = await client.select_time_series("lsst.sal.ESS.accelerometerPSD", ["*"], start, end)

indexCounter = 0
plt.figure(figsize=(8,8))
plt.subplots_adjust(wspace=0.5, hspace=0.5)
M12_axes = ['X', 'Y', 'Z']
truss_axes = ['X', 'Z', 'Y']
sensors = ["AuxTel-M1", "AuxTel-M2", "AuxTel-Truss"]
plotCounter = 1
psd_sum = {}
for sensor in sensors:
    psd_sum[sensor] = {}
    if sensor in ["AuxTel-M1", "AuxTel-M2"]:
        axes = M12_axes
    else:
        axes = truss_axes
    for axis in axes:
        base_field = f"accelerationPSD{axis}"
        trueAxis = trueAxes[sensor][axis]
        plt.subplot(3,3,plotCounter)
        plt.title(f"{sensor} - {trueAxis}", fontsize=12)
        df = merge_packed_PSD(accel_data, base_field, sensor)
        row = df.iloc[indexCounter][2:] * 1E10
        row.plot()
        sum = row.sum()
        #print(sensor, trueAxis, sum)
        psd_sum[sensor][trueAxis] = sum

        plt.xlabel('Frequency [Hz]')
        plt.ylabel('PSD [1E-9 m^2/(Hz s^4)]')
        plt.ylim(0.0, 5.0)
        plotCounter += 1
timestamp = df.index[0].strftime("%Y%m%dT%H%M%SZ")
plt.suptitle(f"Accelerometer Power Spectral Density - {timestamp}", fontsize=16)
#plt.savefig(f"/home/craiglagegit/DATA/Accel_PSD_Scaled_{timestamp}.pdf")

In [ ]:
psd_sum

In [ ]:
# Plotting the mount plots
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", "azimuthCalculatedAngle", start, end)
el = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", "elevationCalculatedAngle", start, end)
# Calculate the tracking errors
az_vals = np.array(az.values[:, 0])
el_vals = np.array(el.values[:, 0])
times = np.array(az.values[:, 1])
# The fits are much better if the time variable
# is centered in the interval
fit_times = times - times[int(len(az.values[:, 1]) / 2)]

# Fit with a polynomial
az_fit = np.polyfit(fit_times, az_vals, 4)
el_fit = np.polyfit(fit_times, el_vals, 4)
az_model = np.polyval(az_fit, fit_times)
el_model = np.polyval(el_fit, fit_times)

# Errors in arcseconds
az_error = (az_vals - az_model) * 3600
el_error = (el_vals - el_model) * 3600

# Calculate RMS
az_rms = np.sqrt(np.mean(az_error * az_error))
el_rms = np.sqrt(np.mean(el_error * el_error))

# Calculate Image impact RMS
image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
image_el_rms = el_rms

In [ ]:
plt.figure(figsize=(8,8))
plt.subplots_adjust(wspace = 0.5, hspace = 0.5)
title = f"Mount Tracking {center_time.isot}"
plt.suptitle(title, fontsize=18)
# Azimuth axis
plt.subplot(2, 2, 1)
ax1 = az['azimuthCalculatedAngle'].plot(legend=True, color='red')
ax1.set_title("Azimuth axis", fontsize=16)
ax1.axvline(az.index[0], color="red", linestyle="--")
#ax1.set_xticks([])
ax1.set_ylabel("Degrees")
plt.subplot(2, 2, 3)
plt.plot(fit_times, az_error, color='red')

plt.title(f"Azimuth RMS error = {az_rms:.2f} arcseconds\n"
          f"  Image RMS error = {image_az_rms:.2f} arcseconds")
plt.ylim(-10.0, 10.0)
plt.xticks([])
plt.ylabel("Arcseconds")

# Elevation axis
plt.subplot(2, 2, 2)
ax2 = el['elevationCalculatedAngle'].plot(legend=True, color='green')
ax2.set_title("Elevation axis", fontsize=16)
ax2.axvline(az.index[0], color="red", linestyle="--")
#ax2.set_xticks([])
plt.subplot(2, 2, 4)
plt.plot(fit_times, el_error, color='green')
plt.title(f"Elevation RMS error = {el_rms:.2f} arcseconds\n"
          f"    Image RMS error = {image_el_rms:.2f} arcseconds")
plt.ylim(-10.0, 10.0)
plt.xticks([])
#plt.savefig(saveFilename)

 

In [ ]:
freqs = np.fft.rfftfreq(el_error.size, 0.001)
psd = np.abs(np.fft.rfft(el_error)) ** 2
plt.subplot(2,1,1)
plt.plot(freqs, psd)
plt.xlim(0,20)
plt.ylim(0, 1E4)
axis = 'X'
sensor = 'AuxTel-Truss'
base_field = f"accelerationPSD{axis}"
trueAxis = trueAxes[sensor][axis]
plt.subplot(2,1,2)
#plt.title(f"{sensor} - {trueAxis}", fontsize=12)
df = merge_packed_PSD(accel_data, base_field, sensor)
row = df.iloc[indexCounter][2:]
row.plot()
plt.xlim(0,20)

